# CNN Example From Flowers by Keras (Color Version)


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models, regularizers

import numpy as np
import matplotlib.pyplot as plt



In [2]:
from os import listdir
from os.path import isfile, join
from pylab import *
from numpy import *

def getFolder(thePath,isFile=True):
    return [f for f in listdir(thePath) if isFile == isfile(join(thePath, f)) ]


def getImagesAndLabels(tPath):
    labels=getFolder(tPath,False)
    encoder={w: i for i, w in enumerate(labels)}
    decoder={i: w  for i, w in enumerate(labels)}

    
    tImages,tLabels=None,None

    for label in labels:
        imgFolder=join(tPath,label)
        files= [f for f in listdir(imgFolder) if isfile(join(imgFolder, f))]
        for file in files:
            imageFile=join(imgFolder,file)
            img=np.array([imread(imageFile)])/255.
            if tImages is None:
                tImages, tLabels =img, np.array([encoder[label]],ndmin=1)
            else:
#                 print(tImages.shape,img.shape,imageFile)
                tImages = np.vstack( (tImages,img) )
                tLabels = np.append(tLabels,np.array([encoder[label]]) ,axis=0) 
        print(label,tImages.shape)                 
    return (tImages,tLabels,encoder,decoder)

    

In [3]:
%%bash
x=$(dpkg -l|grep imagemagick);[ ${#x} -eq 0 ]&&sudo apt install -y imagemagick
[ ! -f ./flower_photos.tgz ]&&wget http://download.tensorflow.org/example_images/flower_photos.tgz ; tar xzf ./flower_photos.tgz 
find ./flower_photos|grep jpg|xargs -i -t convert {} -quiet -resize 300x224\! -quality 100 -colorspace RGB -type truecolor {} 2>> /tmp/log
find ./flower_photos|grep jpg|head -n 10|xargs -i -t identify {}

./flower_photos/daisy/5885826924_38fdc6bcaa_n.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 91.3KB 0.000u 0:00.000
./flower_photos/daisy/12701063955_4840594ea6_n.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 137KB 0.000u 0:00.000
./flower_photos/daisy/5997702776_c7bc37aa6b_n.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 81.7KB 0.000u 0:00.000
./flower_photos/daisy/5673728_71b8cb57eb.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 77.7KB 0.000u 0:00.000
./flower_photos/daisy/11834945233_a53b7a92ac_m.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 77KB 0.000u 0:00.000
./flower_photos/daisy/9204730092_a7f2182347.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 108KB 0.000u 0:00.000
./flower_photos/daisy/7066602021_2647457985_m.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 111KB 0.000u 0:00.000
./flower_photos/daisy/43474673_7bb4465a86.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 76.6KB 0.000u 0:00.000
./flower_photos/daisy/5876455546_32049e5585.jpg JPEG 300x224 300x224+0+0 8-bit sRGB 64.6KB 0.000u 0:00.000
./flower_photos/daisy/512177035_70a



E: 잠금 파일 /var/lib/dpkg/lock-frontend 파일을 열 수 없습니다 - open (13: 허가 거부)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
identify ./flower_photos/daisy/5885826924_38fdc6bcaa_n.jpg 
identify ./flower_photos/daisy/12701063955_4840594ea6_n.jpg 
identify ./flower_photos/daisy/5997702776_c7bc37aa6b_n.jpg 
identify ./flower_photos/daisy/5673728_71b8cb57eb.jpg 
identify ./flower_photos/daisy/11834945233_a53b7a92ac_m.jpg 
identify ./flower_photos/daisy/9204730092_a7f2182347.jpg 
identify ./flower_photos/daisy/7066602021_2647457985_m.jpg 
identify ./flower_photos/daisy/43474673_7bb4465a86.jpg 
identify ./flower_photos/daisy/5876455546_32049e5585.jpg 
identify ./flower_photos/daisy/512177035_70afc925c8.jpg 


In [4]:
from sklearn.model_selection import train_test_split

basePath='./flower_photos'
imgs,labels,encoder,decoder=getImagesAndLabels(basePath)

class_cnt=len(encoder.keys())

train_images,test_images,train_labels,test_labels=train_test_split(imgs,labels,test_size=0.3333, random_state=99)

daisy (633, 224, 300, 3)
tulips (1432, 224, 300, 3)
dandelion (2330, 224, 300, 3)
roses (2971, 224, 300, 3)
sunflowers (3670, 224, 300, 3)


In [5]:
imgShape=train_images[0].shape
img_width,img_hight,color_count=imgShape
train_images = train_images.reshape((-1,img_width,img_hight,color_count ))
test_images = test_images.reshape((-1, img_width,img_hight,color_count ))

print("Shape of Train_images = ",train_images.shape)

Shape of Train_images =  (2446, 224, 300, 3)


In [6]:

model = models.Sequential()
model.add(layers.Conv2D(128, (11, 11), activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.02), input_shape=imgShape))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (7, 7), activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((4, 4)))

model.add(layers.Conv2D(4, (3, 3), activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid',kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dense(class_cnt, activation='softmax'))
# model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
from tensorflow.python.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0002),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=50,batch_size=10)

Train on 2446 samples
Epoch 1/50
2446/2446 [==============================] - 107s 44ms/sample - loss: 2.9514 - acc: 0.3684
Epoch 2/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.6785 - acc: 0.5012
Epoch 3/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.4727 - acc: 0.5384
Epoch 4/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.3461 - acc: 0.5670
Epoch 5/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.2850 - acc: 0.5732
Epoch 6/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.2236 - acc: 0.5998
Epoch 7/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.1904 - acc: 0.6132
Epoch 8/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.1502 - acc: 0.6382
Epoch 9/50
2446/2446 [==============================] - 103s 42ms/sample - loss: 1.1280 - acc: 0.6374
Epoch 10/50
2446/2446 [==============================] - 103

In [8]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.5334567493862576
Test accuracy: 0.5416667


In [ ]:
# Reularization 구혀해 보세요.